In [1]:
import os

In [2]:
%pwd

'd:\\Text-summary\\Text-summary\\Text-summary\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Text-summary\\Text-summary\\Text-summary'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file:Path
    unzip_dir: Path
    
    


In [13]:
from textSummary.constants import *



ModuleNotFoundError: No module named 'textSummary'

In [7]:
import textSummary.utils.common
print(dir(textSummary.utils.common))

['BoxError', 'ConfigBox', 'List', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__warningregistry__', 'create_directories', 'ensure_annotations', 'get_size', 'log_dir', 'log_filepath', 'logger', 'logging', 'logging_str', 'os', 'read_yaml', 'sys', 'yaml']


In [8]:
from textSummary.utils.common import  read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from textSummary.logging import logger
from textSummary.utils.common import get_size


In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

ValueError: Error loading YAML file: mapping values are not allowed here
  in "config\config.yaml", line 3, column 15